In [175]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import yfinance as yf
import scipy.optimize as sc
import plotly.graph_objects as go

In [176]:
def getData(stocks, start, end):
    stockData = yf.download(
        stocks,
        start=start,
        end=end,
        auto_adjust=True   # 👈 ÇOK ÖNEMLİ
    )

    prices = stockData['Close']
    returns = prices.pct_change().dropna()

    meanReturns = returns.mean() * 252   # 👈 annualized
    covMatrix = returns.cov() * 252      # 👈 annualized

    return meanReturns, covMatrix


def portfolioPerformance(weights, meanReturns, covMatrix):
    # meanReturns ve covMatrix zaten yıllık
    portReturn = np.sum(meanReturns * weights)
    portStd = np.sqrt(np.dot(weights.T, np.dot(covMatrix, weights)))
    return portReturn, portStd


stockList = ['CBA.AX', 'BHP.AX', 'TLS.AX']
stocks = [stock for stock in stockList]

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=365)

weights = np.array([0.3, 0.3, 0.4])

meanReturns, covMatrix = getData(stocks, start = startDate, end=endDate)
returns, std = portfolioPerformance(weights, meanReturns, covMatrix)


print(round(returns*100,2), round(std*100,2))

[*********************100%***********************]  3 of 3 completed

18.07 11.36


In [177]:
# Negative sharpe ratio function

def negativeSR(weights, meanReturns, covMatrix, riskFreeRate = 0.04):
  pReturns, pStd = portfolioPerformance(weights,meanReturns, covMatrix)
  return - (pReturns - riskFreeRate)/pStd

def maxSR(meanReturns, covMatrix, riskFreeRate = 0.04, constraintSet = (0,1)):
  'Minimize the negative SR, by altering the weights of the portfolio'
  numAssets = len(meanReturns)
  args = (meanReturns, covMatrix, riskFreeRate)

  constraints = ({'type' : 'eq', 'fun': lambda x: np.sum(x) - 1})
  bounds = tuple(constraintSet for asset in range(numAssets))
  result = sc.minimize(negativeSR, numAssets*[1./numAssets], args = args, method = 'SLSQP', bounds = bounds, constraints = constraints)

  return result


result_max_sr = maxSR(meanReturns, covMatrix)

max_sr_value, maxWeights = result_max_sr['fun'], result_max_sr['x'] # Rename variable to avoid overwriting function
print(result_max_sr)
print(max_sr_value, maxWeights)

     message: Optimization terminated successfully
     success: True
      status: 0
         fun: -1.7956657263438047
           x: [ 2.518e-01  4.131e-17  7.482e-01]
         nit: 6
         jac: [-3.764e-01  4.121e-02 -3.761e-01]
        nfev: 24
        njev: 6
 multipliers: [-3.761e-01]
-1.7956657263438047 [2.51785262e-01 4.13081028e-17 7.48214738e-01]


In [178]:
# Portfolio Optimization with minimum Variance

def portfolioVariance(weights, meanReturns, covMatrix):
  # The portfolioPerformance function already returns std (variance is std^2)
  # We need to return only std for minimization
  return portfolioPerformance(weights, meanReturns, covMatrix)[1]

def minimizeVariance(meanReturns, covMatrix, constraintSet = (0,1)):
  'Minimize the variance, by altering the weights of the portfolio'
  numAssets = len(meanReturns)
  args = (meanReturns, covMatrix)

  constraints = ({'type' : 'eq', 'fun': lambda x: np.sum(x) - 1})
  bounds = tuple(constraintSet for asset in range(numAssets))
  # Minimize the portfolioVariance function directly
  result = sc.minimize(portfolioVariance, numAssets*[1./numAssets], args = args, method = 'SLSQP', bounds = bounds, constraints = constraints)

  return result

# Call minimizeVariance with 'Close' data
minVarResult = minimizeVariance(meanReturns, covMatrix)

min_var_value, minVarWeights = minVarResult['fun'], minVarResult['x'] # Rename variable to avoid overwriting function
print(minVarResult)
print(min_var_value, minVarWeights)

     message: Optimization terminated successfully
     success: True
      status: 0
         fun: 0.1015605695790067
           x: [ 2.190e-01  1.361e-01  6.449e-01]
         nit: 6
         jac: [ 1.015e-01  1.016e-01  1.016e-01]
        nfev: 24
        njev: 6
 multipliers: [ 1.016e-01]
0.1015605695790067 [0.21902767 0.13605289 0.64491944]


In [179]:
def calculatedResults(meanReturns, covMatrix, constraintSet = (0,1)):
  """Read in mean, cov matrix and other financial info
     Output, Max SR, Min Volatility, efficient frontier"""

  # meanReturns and covMatrix are already based on 'Close' prices, no need to re-index
  meanReturns_close = meanReturns
  covMatrix_close = covMatrix


  # Max Sharpe Ratio Portfolio
  maxSR_Portfolio = maxSR(meanReturns_close, covMatrix_close)
  maxSR_Returns, maxSR_std = portfolioPerformance(maxSR_Portfolio['x'], meanReturns_close, covMatrix_close)
  maxSR_allocation = pd.DataFrame(maxSR_Portfolio['x'], index=meanReturns_close.index, columns = ['allocation'])
  maxSR_allocation.allocation = [round(i*100,0) for i in maxSR_allocation.allocation]


  # Min Volatility Portfolio
  minVol_Portfolio = minimizeVariance(meanReturns_close, covMatrix_close)
  minVol_Returns, minVol_std = portfolioPerformance(minVol_Portfolio['x'], meanReturns_close, covMatrix_close)
  minVol_allocation = pd.DataFrame(minVol_Portfolio['x'], index=meanReturns_close.index, columns = ['allocation'])
  minVol_allocation.allocation = [round(i*100,0) for i in minVol_allocation.allocation]

  # Efficient Frontier
  efficientlist = []
  targetReturns = np.linspace(minVol_Returns, maxSR_Returns, 20)
  for target in targetReturns:
    # Append the standard deviation (fun attribute) from the OptimizeResult
    efficientlist.append(efficientopt(meanReturns, covMatrix, target)['fun'])

  maxSR_Returns, maxSR_std = round(maxSR_Returns*100,2), round(maxSR_std*100,2)
  minVol_Returns, minVol_std = round(minVol_Returns*100,2), round(minVol_std*100,2)
  return maxSR_Returns, maxSR_std, maxSR_allocation, minVol_allocation, minVol_Returns, minVol_std, efficientlist,  targetReturns

print(calculatedResults(meanReturns, covMatrix))

(np.float64(23.1), np.float64(10.63),         allocation
Ticker            
BHP.AX        25.0
CBA.AX         0.0
TLS.AX        75.0,         allocation
Ticker            
BHP.AX        22.0
CBA.AX        14.0
TLS.AX        64.0, np.float64(20.81), np.float64(10.16), [np.float64(0.10156064760921397), np.float64(0.10157430160472282), np.float64(0.10161505525842311), np.float64(0.10168287606604794), np.float64(0.10177770999495998), np.float64(0.10189948169004075), np.float64(0.10204809480364625), np.float64(0.10222343232569035), np.float64(0.10242535707995296), np.float64(0.10265371223002352), np.float64(0.10290832189627454), np.float64(0.1031889917563838), np.float64(0.10349550986689102), np.float64(0.10382764733209689), np.float64(0.10418515916297225), np.float64(0.10456778514377892), np.float64(0.10497525061224995), np.float64(0.10540726763832556), np.float64(0.10586300232072204), np.float64(0.10634316855675952)], array([0.20813157, 0.20933289, 0.21053422, 0.21173555, 0.21293688,
    

In [180]:
def portfolioReturn(weights, meanReturns_arg, covMatrix_arg):
  return portfolioPerformance(weights, meanReturns_arg, covMatrix_arg)[0]


def efficientopt(meanReturns, covMatrix, returnTarget, constraintSet = (-1,1)):
  """For each returnTarget, we want to optimize the portfolio min variance"""
  # meanReturns and covMatrix are already based on 'Close' prices, no need to re-index
  numAssets = len(meanReturns)
  args = (meanReturns, covMatrix)

  constraints = ({'type': 'eq', 'fun' : lambda x: portfolioReturn(x, meanReturns, covMatrix)  - returnTarget},
                {'type' : 'eq', 'fun': lambda x: np.sum(x) - 1})
  bounds = tuple(constraintSet for asset in range(numAssets))
  effopt = sc.minimize(portfolioVariance, numAssets*[1./numAssets], args=args, method = 'SLSQP', bounds = bounds, constraints=constraints)

  return effopt

In [181]:
print(efficientopt(meanReturns, covMatrix, 0.10))

     message: Optimization terminated successfully
     success: True
      status: 0
         fun: 0.18061189266955405
           x: [ 6.464e-02  7.806e-01  1.548e-01]
         nit: 4
         jac: [ 3.060e-02  2.227e-01  3.118e-02]
        nfev: 16
        njev: 4
 multipliers: [-1.142e+00  2.948e-01]


In [182]:
def EF_graph(maxSR_Returns, maxSR_std, minVol_Returns, minVol_std, efficientlist, targetReturns, riskFreeRate = 0.04, constraintSet = (0, 1)):
  """Return a graph plotting the min vol, max sr and EF"""
  # Max SR

  MaxSharpeRatio = go.Scatter(
        name = 'Maximum Sharpe Ratio',
       mode = 'markers',
        x=[maxSR_std],
        y=[maxSR_Returns],
      marker = dict(color='red', size = 14, line =dict(width=3, color='black'))
   )

  # Min Vol
  MinVol = go.Scatter(
      name = 'Minimum Volatility',
      mode = 'markers',
        x=[minVol_std],
        y=[minVol_Returns],
      marker = dict(color='green', size = 14, line =dict(width=3, color='black'))
   )

  # Efficient Frontier
  EF_curve = go.Scatter(
        name = 'Efficient Frontier',
       mode = 'lines',
        x=[round(ef_std*100, 2)for ef_std in efficientlist],
        y=[round(target*100, 2) for target in targetReturns],
      line = dict(color='black', dash='dashdot') # Moved dash property here and set mode to 'lines'
   )

  data = [MaxSharpeRatio, MinVol, EF_curve]

  layout = go.Layout(
      title = 'Portfolio Optimization with the Efficient Frontier',
      yaxis_title = 'Annualised Return (%)', # Changed to yaxis_title
      xaxis_title = 'Annualised Volatility (%)', # Changed to xaxis_title
      showlegend= True,
      legend = dict(x = 0.75, y = 0, traceorder = 'normal',
                    bgcolor = '#E2E2E2',
                    bordercolor = 'black',
                    borderwidth = 2),
      width = 800,
      height = 600)
  fig = go.Figure(data = data, layout = layout)
  return fig.show()

# Call calculatedResults to get the necessary data
maxSR_Returns, maxSR_std, maxSR_allocation, minVol_allocation, minVol_Returns, minVol_std, efficientlist, targetReturns = calculatedResults(meanReturns, covMatrix)

# Call EF_graph with the unpacked results
EF_graph(maxSR_Returns, maxSR_std, minVol_Returns, minVol_std, efficientlist, targetReturns)